In [5]:
%load_ext autoreload
%autoreload 2
import ML
import torch
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from tqdm.auto import tqdm
from pathlib import Path

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
def f(x):
    return x[0]


def delete_folder(folder_path):
    folder = Path(folder_path)
    for item in folder.iterdir():
        if item.is_file():
            item.unlink()
        else:
            delete_folder(item)
    folder.rmdir()

In [11]:
try:
    delete_folder("data")
except:
    pass
val_doc, test_doc = [], []
for rand in [0]:
    val_result, test_result, data_nums = [], [], []
    for iteration in tqdm(range(9,10)):
        repeat_results = []
        best = -1000
        for repeat in (range(1)):
            temp = 0
            folder = f"prepared_data/prepared_data-{rand + 1}/Scaled/Chicago_Sketch/95"
            miss = [0.1,0.2,0.3,0.4,0.5,0.6,0.7, 0.8,0.9,0.95]
            data = ML.DataLoader_Me(folder, miss[iteration], batch_size=512, device="cpu")
            train, val, test = data.train, data.val, data.test

            input_size = 3
            hidden_units = 8
            num_layer = 2
            output_size = 1
            epochs = 500
            batch_size = 512
            # learning_rate = np.random.choice(np.arange(0.0001, 0.0011, 0.0001))
            learning_rate = 0.0001

            model = ML.FlowPredict(input_size, hidden_units, output_size)
            count_epoch, loss_values, val_loss_values = ML.train_model(model, train, val, epochs, learning_rate)
            [val_rmse, val_mae, val_r2], data_val = ML.evaluate_model(model, val)
            [test_rmse, test_mae, test_r2], data_test = ML.test_model(model, test)
            if best < test_r2 or repeat == 99:
                # best = test_r2
                best_list = [[model, [count_epoch, loss_values, val_loss_values]],
                             [[val_rmse, val_mae, val_r2], data_val], [[test_rmse, test_mae, test_r2], data_test],
                             [{"input_size": [input_size], "output_size": [output_size], "hidden_units": [hidden_units],
                               "num_hidden_layer": [num_layer], "epochs": [epochs], "batch_size": [batch_size],
                               "learning_rate": [learning_rate], "optimizer": ["Adam"], "loss": ["MSE"],
                               "activation": ["ReLU"]}]]
                break

        model, [count_epoch, loss_values, val_loss_values] = best_list[0]
        [val_rmse, val_mae, val_r2], data_val = best_list[1]
        [test_rmse, test_mae, test_r2], data_test = best_list[2]
        model_hyperparameter = pd.DataFrame(best_list[3][0])

        val_result.append([np.array(val_rmse), np.array(val_mae), val_r2])
        test_result.append([np.array(test_rmse), np.array(test_mae), test_r2])

        print(f"R2 is in{rand + 1}/10 and {iteration + 1}/9 == {test_r2}")
        ML.plot_fn(data_test, save=True, show=False, model=f"Chicago_Sketch{rand}_{iteration}",
                   bins=np.arange(-0.001, 0.0011, 0.0001))

        if not os.path.exists(f"data/Result{rand + 1}/Chicago_Sketch_data"):
            os.makedirs(f"data/Result{rand + 1}/Chicago_Sketch_data")
        model_hyperparameter.to_csv(
            f"data/Result{rand + 1}/Chicago_Sketch_data/HyperParameters{iteration + 1}_NN_model.csv")
        data_test = np.array(data_test)
        if not os.path.exists(f"data/Result{rand + 1}/Chicago_Sketch_data/data_NN_model.csv"):
            data_test = pd.DataFrame({"Predict1": data_test[0].squeeze(), "Real1": data_test[1].squeeze()})
            data_test.to_csv(f"data/Result{rand + 1}/Chicago_Sketch_data/data_NN_model.csv")
        else:
            data_last = pd.read_csv(f"data/Result{rand + 1}/Chicago_Sketch_data/data_NN_model.csv")
            data_last = data_last.iloc[:, 1:]
            data_last[f"Predict{iteration + 1}"] = data_test[0].squeeze()
            data_last[f"Real{iteration + 1}"] = data_test[1].squeeze()
            data_last.to_csv(f"data/Result{rand + 1}/Chicago_Sketch_data/data_NN_model.csv")

        with torch.inference_mode():
            # plt.clf()
            # plt.plot(count_epoch[5:], loss_values[5:], c="b", label="Train")
            # plt.plot(count_epoch[5:], val_loss_values[5:], c="r", label="val")
            # plt.legend()
            # plt.show()
            if not os.path.exists("Models/Chicago_Sketch"):
                os.makedirs("Models/Chicago_Sketch")
            torch.save(model, f"Models/Chicago_Sketch/Chicago_Sketch{rand}_{iteration}_model_nn.pth")

    if not os.path.exists(f"data/Result{rand + 1}/Chicago_Sketch_data/NN_model.csv"):
        test_result = pd.DataFrame(test_result, columns=["RMSE", "MAE", "R^2"])
        test_result.to_csv(f"data/Result{rand + 1}/Chicago_Sketch_data/NN_model.csv")
    else:
        data_last = pd.read_csv(f"data/Result{rand + 1}/Chicago_Sketch_data/NN_model.csv")
        data_last = data_last.iloc[:, 1:]
        test_result = pd.DataFrame(test_result, columns=["RMSE", "MAE", "R^2"])
        test_result = pd.concat([test_result, data_last], ignore_index=True)
        test_result.to_csv(f"data/Result{rand + 1}/Chicago_Sketch_data/NN_model.csv")




  0%|          | 0/1 [00:00<?, ?it/s]

___0.019549___0.018494___
___0.018837___0.017840___
___0.018204___0.017210___
___0.017602___0.016608___
___0.017028___0.016034___
___0.016480___0.015488___
___0.015956___0.014969___
___0.015456___0.014475___
___0.014978___0.014004___
___0.014522___0.013556___
___0.014087___0.013130___
___0.013673___0.012723___
___0.013276___0.012334___
___0.012897___0.011963___
___0.012533___0.011607___
___0.012185___0.011266___
___0.011851___0.010939___
___0.011531___0.010625___
___0.011223___0.010323___
___0.010926___0.010033___
___0.010641___0.009754___
___0.010365___0.009485___
___0.010100___0.009226___
___0.009843___0.008976___
___0.009596___0.008736___
___0.009357___0.008504___
___0.009126___0.008280___
___0.008903___0.008064___
___0.008688___0.007855___
___0.008480___0.007654___
___0.008279___0.007459___
___0.008083___0.007271___
___0.007893___0.007088___
___0.007709___0.006912___
___0.007531___0.006741___
___0.007359___0.006576___
___0.007193___0.006417___
___0.007031___0.006263___
___0.006875_

# Linear Regression

In [13]:
import LinearRegression as Ll

r2 = []
for rand in tqdm(range(10)):
    all_r2 = []
    one = [0, 1, 2]
    two = [3, 4, 5]
    three = [7, 8]
    for iterr in tqdm(range(9)):
        folder = f"prepared_data/prepared_data-{rand + 1}/Scaled/Chicago_Sketch/{int((iterr + 1) * 10)}"
        miss = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
        x_train, y_train, x_val, y_val, x_test, y_test = Ll.load(folder, miss[iterr])
        model = Ll.reg(x_train, y_train)
        r = Ll.testit(model, x_val, y_val)
        r2.append(r)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

...

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]